In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import csv
import pandas as pd 
import nltk
import random
import string
import warnings
import numpy as np
import re
import dask.dataframe as dd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
warnings.filterwarnings('ignore')

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/content.csv', error_bad_lines=False)

In [ ]:
data = data.head(400)

In [ ]:
sent_tokens = data['content'].apply(nltk.sent_tokenize)

In [ ]:
def flat(lis):

    flatList = []

    # Iterate with outer list

    for element in lis:

        if type(element) is list:

            # Check if type is list than iterate through the sublist

            for item in element:

                flatList.append(item)

        else:

            flatList.append(element)

    return flatList

In [ ]:
snt_tkn = flat(sent_tokens)

In [ ]:
snt_tkn = sorted(snt_tkn)

In [ ]:
print(len(snt_tkn))

In [ ]:
lemmer = nltk.stem.WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [ ]:
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up", "hey")
GREETING_RESPONSES = ["hi", "hey", "nods", "hi there", "hello", "I am glad! you are talking to me"]

def greeting(scentence):
    
    for word in scentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

In [ ]:
def bin_search(L, target):
    start = 0
    end = len(L) - 1
    while start <= end:
        middle = (start + end)// 2
        midpoint = L[middle]
        if midpoint > target:
            end = middle - 1
        elif midpoint < target:
            start = middle + 1
        else:
            return midpoint

In [ ]:
def response(user_response):
    chatbot_response = ''
    data.loc[len(data.index)] = [user_response]
    # snt_tkn.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words="english")
    tfidf = TfidfVec.fit_transform(data['content'])
    # tfidf = TfidfVec.fit_transform(snt_tkn)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx = vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf == 0):
        chatbot_response = chatbot_response + "I am sorry! I don't understand you"
        return chatbot_response
    
    else:
        chatbot_response=chatbot_response+data['content'][idx]
        return chatbot_response

In [ ]:
if __name__ == "__main__":
    flag = True
    print("Hello, there my name is Aneka. I will answer your queries. If you want to exit, type Bye!")
    while(flag==True):
        user_response = input('You: ')
        user_response = user_response.lower()
        if(user_response!='bye'):
            if user_response == 'thanks' or user_response == 'thank you':
                flag = False
                print("Bot: You're welcome!")
            else:
                if(greeting(user_response)!=None):
                    print("Bot:" +greeting(user_response))
                else:
                    print("Bot:", end='')
                    print(response(user_response))
                    # snt_tkn.remove(user_response)
                    data = data[:-1]
                    
        else:
            flag = False
            print("Bot: Bye! Have a great time!" )